Installing packages for operations. Pandas for storing data, random for random split, numpy for doing operations in numpy

In [ ]:
import pandas as pd 
import random 
import numpy as np

Working direction for saving files for later

In [ ]:
import os 
os.getcwd()

Installing the ucimlrepo for getting the data

In [ ]:
pip install ucimlrepo 

Loading the Adult Census Income dataset from the ucirepo

In [ ]:
## Loading the ADULT dateset from UCI repo 
#Avaliable at https://archive.ics.uci.edu/dataset/2/adult
import ucimlrepo
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features # Importing all features 
y = adult.data.targets # Importing target varialbes

Recoding the y variable to match. Specifically removing the '.'

In [ ]:
y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})

Checking only one instance of each outcome of the y varaible 

In [ ]:
y.value_counts()

Concatenating both x and y together in a dataframe

In [ ]:
data =  pd.concat([X, y], axis=1)

Inspecting the datatypes for any unexpected data formats

In [ ]:
# Checking the data types of all the columns
data.dtypes

Recoding the y variable to binary

In [ ]:
## Converting the Predictor Variable into Numeric 
data['income']=data['income'].map({'<=50K':0,'>50K':1})
data['income'].value_counts()

Checking the number ? in the dataframe 

In [ ]:
# Count occurrences of x in each column
for col in data:
    count = (data[col] == '?').sum()
    print(f"Number of values equal to {'?'} in column {col}: {count}")

Recoding ? to na

In [ ]:
data[data == '?'] = np.nan

Checking number of na in each columns

In [ ]:
data.isna().sum()

Dropping na's 

In [ ]:
data = data.dropna()

Checking the distribution of the y varialbe

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the distribution of the 'income' variable
plt.figure(figsize=(8, 6))
sns.countplot(x='income', data=data, palette='viridis')
plt.title('Distribution of Income')
plt.xlabel('Income (0: <=50K, 1: >50K)')
plt.ylabel('Count')
plt.show()

Plotting the correlation matrix for the numerical features

In [ ]:
# Identify numerical features
numerical_features = data.select_dtypes(include=['number'])

# Compute the correlation matrix for numerical features only
correlation_matrix = numerical_features.corr()

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix for Numerical Features')
plt.show()

Checking the distribution of all categorical variables

In [ ]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns

# Plotting the distribution of each categorical feature
for feature in categorical_features:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=feature, data=data, palette='viridis')
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

Checking the education with the education numerical column, to see their if they are redundant 

In [ ]:
pd.crosstab(
    index=data["education"], columns=data["education-num"]
)

Checking the procentage of the majority class in the native-country

In [ ]:
# Calculate the percentage of 'United States' in 'native country' column
total_count = len(data)
us_count = data['native-country'].value_counts().get('United-States', 0)
percentage_us = (us_count / total_count) * 100
percentage_us

Removing outlier classes in selected features

In [ ]:
data = data[data['workclass'] != 'Without-pay']

In [ ]:
data = data[data['education'] != 'Preschool']

In [ ]:
data = data[data['occupation'] != 'Armed-Forces']

Recoding material status and removing other classes

In [ ]:
def map_marital_status(status):
    if status in ['Never-married', 'Married-civ-spouse']:
        return status
    elif status in ['Divorced', 'Separated']:
        return 'Divorced or Separated'
    else:
        return None

# Apply the function to the 'marital-status' column
data['marital-status'] = data['marital-status'].map(map_marital_status)

# Drop rows with None values in 'marital-status' column
df = data.dropna(subset=['marital-status'])

Dropping columns based on redundancy and near zero variance 

In [ ]:
data = data.drop(columns=['education-num'])

In [ ]:
data = data.drop(columns=['native-country'])

Checking the new distributions

In [ ]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns

# Plotting the distribution of each categorical feature
for feature in categorical_features:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=feature, data=data, palette='viridis')
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

One hot encoding of categorical features

In [ ]:
data = pd.get_dummies(data, drop_first = False, dummy_na=True)

Splitting the data into test and train

In [ ]:
from sklearn.model_selection import train_test_split

y = data['income'].values
features = [col for col in data.columns if col not in ['income']]
X = data[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3, stratify=y)

Running the Random Forrest Model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

param_grid = {
    'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'max_features': ['sqrt'], #['sqrt'],
    'min_samples_leaf': [10, 15, 20, 25, 30],
    'min_samples_split': [5, 10, 15, 20],
    'n_estimators': [10, 15, 20, 30, 40, 50]
    }

# Initialize the RandomForestClassifier (you can choose any other classifier)
classifier = RandomForestClassifier(random_state=0)

# Train the classifier on the training set
n_estimators = 100
grid_search_rf = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_params_rf = grid_search_rf.best_params_
print("Best Hyperparameters:", best_params_rf)


# Use the best model for predictions
best_model_rf = grid_search_rf.best_estimator_
y_pred_rf = best_model_rf.predict(X_test)
y_pred_rf_df = pd.DataFrame(best_model_rf.predict(X_test), columns=['Predictions'])

Inspecting the accurary

In [ ]:
#Print accuracy and classification report
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy_rf)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.inspection import permutation_importance

# Calculate feature importances
#result_rf = permutation_importance(best_model_rf, X_test, y_test, n_repeats=10, random_state=42)

# Get sorted indices of feature importances
#sorted_idx_rf = result_rf.importances_mean.argsort()[-10:]  # Selecting the top 10 most important features

# Plot
#fig, ax = plt.subplots()
#ax.boxplot(result_rf.importances[sorted_idx_rf].T, vert=False, labels=np.array(sorted_idx_rf)+1)
#ax.set_title("Top 10 Variable Importance in Projection (Random Forest)")
#ax.set_ylabel("Features")
#fig.tight_layout()
#plt.show()


PLotting feature importance

In [ ]:
# Get feature importances from the best model
feature_importances = best_model_rf.feature_importances_

# Create a DataFrame with feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort the features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Select the top 10 features
top_10_features = feature_importance_df.head(25)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=top_10_features, palette='viridis')
plt.title('Top 10 Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.inspection import permutation_importance

# Get feature importances from the best model
permutation_feature_importances = permutation_importance(best_model_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)

# Create a DataFrame with feature names and their importances
permutation_importances = pd.Series(permutation_feature_importances.importances_mean, index=X_train.columns)

# Select the top 10 highest feature importances
top_10_importances = permutation_importances.nlargest(25)

# Plot using seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_importances.values, y=top_10_importances.index, palette='viridis')
plt.title("Top 10 Feature Importances (Random Forest)")
plt.xlabel("Mean Decrease in Impurity")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

Getting the proberbilities of the first obsevation for user test

In [ ]:
# Get the first observation from the test set
first_observation = X_test.iloc[[0]]

# Predict the class label for the first observation
prediction = best_model_rf.predict(first_observation)

# Get the predicted probabilities for each class for the first observation
probabilities = best_model_rf.predict_proba(first_observation)

# Print the prediction and probabilities
print("Prediction:", prediction)
print("Probabilities:")
for class_index, class_probability in enumerate(probabilities[0]):
    print(f"Class {best_model_rf.classes_[class_index]}: {class_probability:.2f}")


Creating a copy of the base data

In [ ]:
basedata = X_test.copy()

Attaching the actual class, prediction and proberbility of the predicted class the the basedata dataframe

In [ ]:
basedata['Actual Class'] = y_test
basedata['Prediction'] = y_pred_rf


# Add a column for the probability of the predicted class
y_pred_proba_rf = best_model_rf.predict_proba(X_test)
predicted_class_probs = [probs[class_index] for probs, class_index in zip(y_pred_proba_rf, y_pred_rf)]
basedata['Predicted Class Probability'] = predicted_class_probs

Check for any missing values

In [ ]:
# Check for missing values
if basedata.isna().values.any():
    print("There are missing values in the DataFrame.")
else:
    print("There are no missing values in the DataFrame.")

Saving to csv for the LLM 

In [ ]:
basedata.to_csv('basedata.csv', index=False)

## Model explainability - Local and Global interpretations

Installing the SHAP package

In [ ]:
pip install shap

Creating the explainer object

In [ ]:
import shap

# Create an explainer for the best model
explainer_rf = shap.Explainer(best_model_rf)

shap_values_rf = explainer_rf(X_test)

Saving the SHAP values to a dataframe

In [ ]:
# Showing a dataframe of each - 0.26 is the base value for churn = 1, for all observations. Each variable then has a SHAP-value +- the BV
shap_df = pd.DataFrame(
    np.c_[shap_values_rf[:, :, 1].base_values, shap_values_rf[:, :, 1].values],
    columns = ["Income"] + list(X_test.columns)
)

Inspecting the SHAP dataframe

In [ ]:
shap_df.head()

Removing the income from the dataframe for the LLM to interpret the SHAP values onlyy

In [ ]:
shap_df_noincome = shap_df.copy()
shap_df_noincome.drop(columns=['Income'], inplace = True)

shap_df_percentage_noincome = shap_df_percentage.copy()
shap_df_percentage_noincome.drop(columns=['Income'], inplace = True)

Saving SHAP value for the LLM 

In [ ]:
shap_df_noincome.to_csv('shap_noincome.csv', index=False)  # Set index=False to not write row numbers as the first column
shap_df_percentage_noincome.to_csv('shap_percentage_noincome.csv', index=False)  # Set index=False to not write row numbers as the first column

# Check of Global Understanding of the model

In [ ]:
shap_sum = np.abs(shap_values_rf[:, :, 1].values).mean(axis=0)

In [ ]:
# Create a dataframe for better readability
shap_importance = pd.DataFrame(list(zip(X.columns, shap_sum)), columns=['Feature', 'SHAP Value'])
shap_importance = shap_importance.sort_values(by='SHAP Value', ascending=False)

In [ ]:
# Print the global SHAP values
pd.options.display.float_format = '{:.6f}'.format
shap_importance.head(10)